In [ ]:
import librosa, itertools, os, sys, numpy as np
from pydub import AudioSegment
from random import shuffle

In [ ]:
dir = # where the solo dataset is
newdir = # where to save the new bpm_mixed dataset
instr = [dir+d for d in os.listdir(dir) if '-' not in d]

In [ ]:
def bpm_sync(f1,f2,d):

    wav,sr = librosa.load(f1,mono=True,sr=22050)
    bpm_base, _ = librosa.beat.beat_track(y=wav, sr=sr)
    # print(f1,"- Initial BPM:",round(bpm_base,3))

    wav,sr = librosa.load(f2,mono=True,sr=22050)
    bpm, _ = librosa.beat.beat_track(y=wav, sr=sr)
    wav = librosa.effects.time_stretch(wav, bpm_base/bpm)
    # bpm, _ = librosa.beat.beat_track(y=wav, sr=sr)
    # print(f2,"- Custom  BPM:",round(bpm,3))

    # adjust to 3 sec
    if len(wav)>3*sr: wav = wav[:3*sr]
    else: wav = np.pad(wav,(0,3*sr-len(wav)),'wrap')
    librosa.output.write_wav(d+"temp_file.wav",wav,sr)

    return d+"temp_file.wav"

In [ ]:
for (instr1, instr2) in list(itertools.combinations(instr,2)):
    
    list1 = [instr1+'/'+f for f in os.listdir(instr1) if "shift" not in f]
    list2 = [instr2+'/'+f for f in os.listdir(instr2) if "shift" not in f]
    shuffle(list1); shuffle(list2)

    if len(list1)<=len(list2): list_b, list_s = list2, list1
    else: list_b, list_s = list1, list2

    folder = instr1[-3:]+'-'+instr2[-3:]
    os.makedirs(newdir+folder,exist_ok=True)

    for i in range(len(list_s)):
        synced_b = bpm_sync(list_s[i],list_b[i],newdir+folder)
        sound1 = AudioSegment.from_file(list_s[i])
        sound2 = AudioSegment.from_file(synced_b)
        mix = sound1.overlay(sound2)
        mix.export(newdir + folder + '/' + folder + '_' + str(i+1) + '.wav', format='wav')
    print("Done with folder:",folder)